# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

624

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# create empty lists to store data for dataframe
city_list = []
latitude = []
temp = []
humidity = []
cloudiness = []
wind_speed = []

In [4]:
#create query url for loop
units = 'metric'
query_url = f'http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={units}&q='

In [5]:
test_response = requests.get(query_url + 'houston').json()
test_response['wind']['speed']

1.79

In [6]:
print('Beginning Data Retrieval')
print('-----------------------------')

#create query url for loop
query_url = f'http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={units}&q='

# loop through cities to collect currect weather data
for city in cities[:25]:
    # Create search query, make request and store in json
    response_json = requests.get(query_url + city).json()
    
    # attempt to grab weather data from the cities
    try:
        latitude.append(response_json['coord']['lat'])
        temp.append(response_json['main']['temp'])
        humidity.append(response_json['main']['humidity'])
        cloudiness.append(response_json['clouds']['all'])
        wind_speed.append(response_json['wind']['speed'])
        city_list.append(city)
        #mass.append(response_json['results'][0]['mass'])
        print(f'Processing Record {cities.index(city)} of Set 1 | {city}')
        
    # exceptions for a cities not found or errors
    except:
        print(f'{city} not found. Skipping.')
        pass

Beginning Data Retrieval
-----------------------------
Processing Record 0 of Set 1 | teya
Processing Record 1 of Set 1 | hithadhoo
Processing Record 2 of Set 1 | port alfred
Processing Record 3 of Set 1 | saskylakh
Processing Record 4 of Set 1 | ribeira grande
Processing Record 5 of Set 1 | barrow
Processing Record 6 of Set 1 | nuuk
Processing Record 7 of Set 1 | ziro
Processing Record 8 of Set 1 | tuy hoa
Processing Record 9 of Set 1 | grand gaube
Processing Record 10 of Set 1 | tiznit
Processing Record 11 of Set 1 | port elizabeth
Processing Record 12 of Set 1 | praia da vitoria
samusu not found. Skipping.
Processing Record 14 of Set 1 | tiksi
Processing Record 15 of Set 1 | avarua
Processing Record 16 of Set 1 | rikitea
Processing Record 17 of Set 1 | cidreira
Processing Record 18 of Set 1 | georgetown
Processing Record 19 of Set 1 | chicama
Processing Record 20 of Set 1 | bermejo
Processing Record 21 of Set 1 | bereda
Processing Record 22 of Set 1 | puerto ayora
Processing Record 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [8]:
# Create DataFrame
weather_data = pd.DataFrame({
    'City': city_list,
    'Latitude': latitude,
    'Temperature (F)': temp,
    'Humidity (%)': humidity,
    'Cloudiness (%)': cloudiness,
    'Wind Speed (mph)': wind_speed
})

weather_data
#weather_data = weather_data.set_index('City')

,City,Latitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,teya,60.3778,24.73,42,89,1.06
1,hithadhoo,-0.6000,27.95,69,100,0.95
2,port alfred,-33.5906,15.44,49,90,4.08
3,saskylakh,71.9167,20.35,40,98,6.16
4,ribeira grande,38.5167,20.80,72,68,4.43
5,barrow,71.2906,4.01,85,90,4.12
6,nuuk,64.1835,6.34,93,75,2.57
7,ziro,27.6333,18.41,89,92,0.20
8,tuy hoa,13.0833,34.87,47,100,5.00
9,grand gaube,-20.0064,19.65,88,67,0.89


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression